<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_vectors_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_vectors_web_lg')


In [ ]:
!mkdir ./results

In [ ]:
!python /content/drive/MyDrive/KAN/train.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/KAN/train.py", line 2, in <module>
    from core.exec import Execution
  File "/content/drive/.shortcut-targets-by-id/1qq5LCjZ3AGmL2Kv20y4s0nEVp9K7G-Mn/KAN/core/exec.py", line 2, in <module>
    from core.model.net import Net
ModuleNotFoundError: No module named 'core.model.net'


In [ ]:
!mkdir ./data
!cp /content/drive/MyDrive/data_path.zip ./data/
!cd ./data && unzip data_path.zip

In [ ]:
!cd ./data && unzip data_path.zip

In [ ]:
# !ls /content/data/data/ques_embeddings/bluebert
!cp -r /content/data/data/ques_embeddings/w2v /content/drive/MyDrive/VQA_path/ques_embeddings

In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy

/content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy


In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo | wc -l

32761


In [ ]:
!ls /content/data/data/ques_embeddings/bioelmo/7563.npy

/content/data/data/ques_embeddings/bioelmo/7563.npy


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

In [ ]:
class Image_Question_Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding,
                 rate=0.1 ,pretrained_cnn_type='inception', multiplier = 10):
        super(Image_Question_Encoder, self).__init__()
        self.maximum_position_encoding = maximum_position_encoding
        self.pretrained_CNN = pretrained_cnn(pretrained_cnn_type)
        self.fc1 = Dense(maximum_position_encoding * multiplier) # fully connected layer
        self.image_dense = Dense(d_model, activation='relu')
        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=maximum_position_encoding)
        self.concatenator = Concatenate(axis=1)
        self.conv = tf.keras.layers.Conv2D(1024, 3, padding='same', activation='relu')
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True))

    def call(self, question, img, training, mask):
        img_feature = self.pretrained_CNN(img)  # (7, 7, 1024)
        img_feature = self.conv(img_feature)
        img_shape = img_feature.shape[-1]
        img_feature = tf.keras.layers.Reshape((-1, img_shape))(img_feature)  # (49, 1024)
        img_feature = self.fc1(img_feature)  # (49, pe_input)
        img_feature = tf.transpose(img_feature, perm=(0, 2, 1))  # (Res, 49)
        img_feature = tf.keras.layers.Reshape((self.maximum_position_encoding, -1))(img_feature)
        img_feature = self.image_dense(img_feature)  # (pe_input, 512)
        question = self.bilstm(question)
        context = self.question_encoder(question, img_feature, training, mask)  # (batch, question_len, 512)
        # context = self.concatenator([img_feature, question_feature]) # （batch, 64 + question_len, 512)
        return context  # (64, 108, 512)

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

# Download and unzip data

In [ ]:
!cp /content/drive/MyDrive/files/data_path.zip /content
!unzip data_path.zip "data/jsons/*" "data/pic/*" "data/ques_embeddings/bioelmo/*" "data/answer_word_frequency.csv" "data/answer_word_frequency.xlsx" -d /content

# Object detection

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# test installation
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
!cp /content/models/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb /content/drive/MyDrive/Colab\ Notebooks

# Entity extraction

In [ ]:
# The textual entities are KG
# entities associated with key phrases of the question, where we utilize the Stanford NLP Dependency Parser [4] and Stanford Named
# Entity Recognizer [5] to analyze the question and extract all noun
# phrases, verb-object phrases and named entities as the key phrases.


In [3]:
from sklearn.utils import shuffle
import pandas as pd
import pathlib

root = pathlib.Path('/content/data')
d = shuffle(pd.read_json(root.joinpath('jsons/open_ended').with_suffix('.json')), random_state=1)

In [4]:
dq = d['Questions']
print(dq)

12134                                     What is present?
9969                                      What is present?
3785                                     What is present ?
2947                            What does this image show?
858                     What are colors of these hormones?
                               ...                        
7813                                      What is present?
10955                           What does this image show?
5192                            Where does this belong to?
12172                                     What is present?
235      How many copies does the red probe hybridize t...
Name: Questions, Length: 16429, dtype: object


In [5]:
print(dq[0])

What are positively charged,  thus allowing the compaction of the negatively charged DNA?


In [6]:
pip install stanfordcorenlp

In [7]:
import spacy
nlp = spacy.load("en_core_web_sm")

def ectract_query(ques):
  qs = nlp(ques)
  queries = []
  for tk in qs:
    # print(tk.text,'|',tk.pos_)
    if tk.pos_ == 'VERB' or tk.pos_ == 'NOUN':
      queries.append(tk.lemma_)
  return queries


print(ectract_query(dq[0]))

['charge', 'allow', 'compaction', 'charge', 'dna']


In [45]:
def knowledge_retrieval(queries):
  kns = []
  for q in queries:
    kns += query_knowledge(q)
  return(kns)

# queries = ectract_query(dq[0])
# query_knowledge

# Knowledge Embeddings

In [14]:
pip install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 870kB 20.1MB/s 
     |████████████████████████████████| 3.3MB 26.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=cb321d575829d9c10c852b06b462063893a9d333f9fb45e8ef5f0cea59ae586f
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [15]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import json
import re
import os

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased", do_lower_case=True)
model = AutoModel.from_pretrained("bert-large-cased", output_hidden_states=True)

with open('/content/data/jsons/QA.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


def preprocess_sentence(w):
    w = w.lower().strip()
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z ]", "", w)
    w = w.strip()
    return w


path = '/content/data/knowledge_embeddings'
if not os.path.exists(path):
    os.mkdir(path)


In [65]:
# What are positively charged,  thus allowing the compaction of the negatively charged DNA?
# what are positively charged thus allowing the compaction of the negatively charged dna
# [CLS] what are positively charged thus allowing the compaction of the negatively charged dna [SEP]
# ['[CLS]', 'what', 'are', 'positively', 'charged', 'thus', 'allowing', 'the', 'compact', '##ion', 'of', 'the', 'negatively', 'charged', 'd', '##na', '[SEP]']
# [101, 1184, 1132, 14257, 4601, 2456, 3525, 1103, 10114, 1988, 1104, 1103, 21598, 4601, 173, 1605, 102]
# [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# tensor([[  101,  1184,  1132, 14257,  4601,  2456,  3525,  1103, 10114,  1988,  1104,  1103, 21598,  4601,   173,  1605,   102]])
# tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

for qa in data:
    ques_id = qa['Question_Id']
    ques = qa['Questions']    

    # extract verbs and nouns from question
    queries = ectract_query(ques)
    # retrieve knowledge from knowledge base
    # num: len(queries) * 5 
    kns = knowledge_retrieval(queries)

    kn_emds = []
    for k in kns:
      marked_text1 = "[CLS] " + k + " [SEP]"    
      tokenized_text1 = tokenizer.tokenize(marked_text1)    
      indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)    
      segments_ids1 = [1] * len(tokenized_text1)    
      tokens_tensor1 = torch.tensor([indexed_tokens1])
      segments_tensors1 = torch.tensor([segments_ids1])

      # tuple - shape: 25 * (1,17,1024) - (batch size,sentence length,embedding size)
      with torch.no_grad():
          # Because we set 'output_hidden_states = True', the third item will be the
          # hidden states from all layers.
          outputs1 = model(tokens_tensor1, segments_tensors1)
          hidden_states1 = outputs1[2]
      # Concatenate the tensors for all layers.
      # Use 'stack' to create a new dimension in the tensor.
      # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
      token_embeddings1 = torch.stack(hidden_states1, dim=0)
      # Remove dimension 1, the "batches".
      # shape: (25,17,1024) - (layers,sentence length,embedding size)
      token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
      # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
      # shape: (17,25,1024) - (sentence length,layers,embedding size)
      token_embeddings1 = token_embeddings1.permute(1, 0, 2)

      token_vecs_sum1 = []

      for token in token_embeddings1:
          # Sum the vectors from the last four layers.
          sum_vec1 = torch.sum(token[-4:], dim=0)
          token_vecs_sum1.append(sum_vec1)
      # shape: (17,1024)
      token_vec = np.array([t.numpy() for t in token_vecs_sum1])
      kn_emds.append(token_vec)
    kn_emds = np.concatenate(kn_emds)
    # Write .npy file to '/content/embedding/' folder.
    np.save(os.path.join(path, str(ques_id)+'.npy'), kn_emds)
    print("saved file: ", ques_id)
print("saved all")

# # validate saved features and calculate max length of all questions
# max_len = 0
# for pair in data:
#     ques_id = pair['Question_Id']
#     emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
#     length = emb.shape[0]
#     if length >= max_len:
#         max_len = length
#         print('current max length is', max_len)

25
saved file:  1
15
saved file:  2
35
saved file:  3
20
saved file:  4
10
saved file:  5
25
saved file:  6
30
saved file:  7
5
saved file:  8
30
saved file:  9
15
saved file:  10
30
saved file:  11
25
saved file:  12
35
saved file:  13
25
saved file:  14
30
saved file:  15
10
saved file:  16
43
saved file:  17
45
saved file:  18
20
saved file:  19
20
saved file:  20
30
saved file:  21
40
saved file:  22
15
saved file:  23
20
saved file:  24
25
saved file:  25
20
saved file:  26
40
saved file:  27
40
saved file:  28
20
saved file:  29
25
saved file:  30
55
saved file:  31
50
saved file:  32
30
saved file:  33
30
saved file:  34
25
saved file:  35
40
saved file:  36
5
saved file:  37
15
saved file:  38
15
saved file:  39
5
saved file:  40
15
saved file:  41
20
saved file:  42
15
saved file:  43
10
saved file:  44
20
saved file:  45
20


KeyboardInterrupt: ignored

In [73]:
# validate saved features and calculate max length of all questions
for pair in data:
  try:
    ques_id = pair['Question_Id']
    emb = np.load('/content/data/knowledge_embeddings/'+str(ques_id)+'.npy')
    # length = emb.shape[0]
    print(ques_id,'shape is', emb.shape)        
  except:
    None

1 shape is (9, 1024)
2 shape is (5, 1024)
3 shape is (9, 1024)
4 shape is (5, 1024)
5 shape is (7, 1024)
6 shape is (7, 1024)
7 shape is (7, 1024)
8 shape is (8, 1024)
9 shape is (8, 1024)
10 shape is (8, 1024)
11 shape is (6, 1024)
12 shape is (6, 1024)
13 shape is (6, 1024)
14 shape is (7, 1024)
15 shape is (7, 1024)
16 shape is (6, 1024)
17 shape is (7, 1024)
18 shape is (10, 1024)
19 shape is (10, 1024)
20 shape is (10, 1024)
21 shape is (10, 1024)
22 shape is (10, 1024)
23 shape is (7, 1024)
24 shape is (7, 1024)
25 shape is (7, 1024)
26 shape is (7, 1024)
27 shape is (10, 1024)
28 shape is (8, 1024)
29 shape is (6, 1024)
30 shape is (8, 1024)
31 shape is (10, 1024)
32 shape is (10, 1024)
33 shape is (8, 1024)
34 shape is (8, 1024)
35 shape is (8, 1024)
36 shape is (8, 1024)
37 shape is (8, 1024)
38 shape is (8, 1024)
39 shape is (8, 1024)
40 shape is (6, 1024)
41 shape is (6, 1024)
42 shape is (6, 1024)
43 shape is (9, 1024)
44 shape is (8, 1024)
45 shape is (9, 1024)


## BERT embeddings

In [43]:
for qa in data:
    ques_id = qa['Question_Id']
    # What are positively charged,  thus allowing the compaction of the negatively charged DNA?
    ques = qa['Questions']
    # what are positively charged thus allowing the compaction of the negatively charged dna
    after = preprocess_sentence(ques)
  
    # [CLS] what are positively charged thus allowing the compaction of the negatively charged dna [SEP]
    marked_text1 = "[CLS] " + after + " [SEP]"
    # ['[CLS]', 'what', 'are', 'positively', 'charged', 'thus', 'allowing', 'the', 'compact', '##ion', 'of', 'the', 'negatively', 'charged', 'd', '##na', '[SEP]']
    tokenized_text1 = tokenizer.tokenize(marked_text1)
    # [101, 1184, 1132, 14257, 4601, 2456, 3525, 1103, 10114, 1988, 1104, 1103, 21598, 4601, 173, 1605, 102]
    indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    segments_ids1 = [1] * len(tokenized_text1)
    # tensor([[  101,  1184,  1132, 14257,  4601,  2456,  3525,  1103, 10114,  1988,  1104,  1103, 21598,  4601,   173,  1605,   102]])
    tokens_tensor1 = torch.tensor([indexed_tokens1])
    # tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
    segments_tensors1 = torch.tensor([segments_ids1])

    # tuple - shape: 25 * (1,17,1024) - (batch size,sentence length,embedding size)
    with torch.no_grad():
        # Because we set 'output_hidden_states = True', the third item will be the
        # hidden states from all layers.
        outputs1 = model(tokens_tensor1, segments_tensors1)
        hidden_states1 = outputs1[2]
    # Concatenate the tensors for all layers.
    # Use 'stack' to create a new dimension in the tensor.
    # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
    token_embeddings1 = torch.stack(hidden_states1, dim=0)
    # Remove dimension 1, the "batches".
    # shape: (25,17,1024) - (layers,sentence length,embedding size)
    token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
    # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
    # shape: (17,25,1024) - (sentence length,layers,embedding size)
    token_embeddings1 = token_embeddings1.permute(1, 0, 2)

    token_vecs_sum1 = []

    for token in token_embeddings1:
        # Sum the vectors from the last four layers.
        sum_vec1 = torch.sum(token[-4:], dim=0)
        token_vecs_sum1.append(sum_vec1)
    # shape: (17,1024)
    token_vec = np.array([t.numpy() for t in token_vecs_sum1])

    
#     # Write .npy file to '/content/embedding/' folder.
#     np.save(os.path.join(path, str(ques_id)+'.npy'), token_vec)
#     print("saved file: ", ques_id)
# # print("saved all")

# # validate saved features and calculate max length of all questions
# max_len = 0
# for pair in data:
#     ques_id = pair['Question_Id']
#     emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
#     length = emb.shape[0]
#     if length >= max_len:
#         max_len = length
#         print('current max length is', max_len)
    break

(17, 1024)


# Knowledge

## SenticNet

In [ ]:
pip install senticnet

     |████████████████████████████████| 51.9MB 142kB/s 


In [ ]:
from senticnet.senticnet import SenticNet

sn = SenticNet()
concept_info = sn.concept('love')
polarity_label = sn.polarity_label('love')
polarity_value = sn.polarity_value('love')
moodtags = sn.moodtags('love')
semantics = sn.semantics('love')
sentics = sn.sentics('love')

In [ ]:
print(concept_info)
print(polarity_label)
print(polarity_value)
print(moodtags)
print(semantics)
print(sentics)

{'polarity_label': 'positive', 'polarity_value': '0.83', 'sentics': {'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}, 'moodtags': ['#joy', '#eagerness'], 'semantics': ['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']}
positive
0.83
['#joy', '#eagerness']
['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']
{'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}


## ConceptNet

In [9]:
pip install conceptnet-lite

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 870kB 15.8MB/s 
  Created wheel for peewee: filename=peewee-3.14.4-cp37-cp37m-linux_x86_64.whl size=593004 sha256=85b782b5e76b37288ec0b74d0dd2b06269f984a39f77eb71dbf6d6b12d1b3e16
  Stored in directory: /root/.cache/pip/wheels/a0/ce/ff/732bd2a2729233b3a55bb5d8557d500f7be41bd21df3f01048
  Created wheel for lmdb: filename=lmdb-0.97-cp37-cp37m-linux_x86_64.whl size=219764 sha256=97a713f15411d83842220aebc2834afe1330f1b80b2471f01bfb21f2ddf0c872
  Stored in directory: /root/.cache/pip/wheels/7f/4d/44/b299a9b22d8eaaaa455304687fb6d4c02d9178c1530850e12a
Successfully built peewee lmdb
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99


In [10]:
import conceptnet_lite

conceptnet_lite.connect("/path/to/conceptnet.db")

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


File not found: /path/to/conceptnet.db
Download compressed database
[*] 0 bytes / 1.85 GB @ 0 bytes/s [------------------] [0.0%, 0s left]    [*] 1.5 MB / 1.85 GB @ 7.7 MB/s [------------------] [0.1%, 0s left]    [*] 14.5 MB / 1.85 GB @ 48.3 MB/s [------------------] [0.8%, 0s left]    [*] 27.5 MB / 1.85 GB @ 68.8 MB/s [------------------] [1.5%, 0s left]    [*] 40.2 MB / 1.85 GB @ 80.4 MB/s [------------------] [2.1%, 0s left]    [*] 53.6 MB / 1.85 GB @ 89.3 MB/s [------------------] [2.8%, 0s left]    [*] 66.7 MB / 1.85 GB @ 95.3 MB/s [-------------

In [ ]:
from conceptnet_lite import Label

cat_concepts = Label.get(text='charge', language='en').concepts  
for c in cat_concepts:
    print("    Concept URI:", c.uri)
    print("    Concept text:", c.text)
    print("    Concept language:", c.language.name)

In [11]:
from conceptnet_lite import Label, edges_for

def query_knowledge(q):
  d = {}
  for e in edges_for(Label.get(text=q, language='en').concepts, same_language=True):
    # print(e.start.text, "::", e.end.text, "|", e.relation.name, "|", e.etc['weight'])
    d[(e.start.text,e.relation.name,e.end.text)] = e.etc['weight']
    # print(e.etc['weight'])
  kn = []
  for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
    if i == 5:
      break
    k = ' '.join(f[0])
    k = k.replace('_', ' ')
    kn.append(k)
  return kn

print(query_knowledge('dna'))

['dna is a molecule', 'dna at location cell', 'dna has property short for deoxyribonucleic acid', 'dna is a complex molecule', 'dna is a double stranded molecule']


In [ ]:
for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
  if i == 5:
    break
  print(f)
  print(f[0])